# GBM Geometry Demo
## J. Michael Burgess

**gbmeometry** is a module with routines for handling GBM geometry. It performs a few tasks:
* creates and astropy coordinate frame for Fermi GBM given a quarternion
* allows for coordinate transforms from Fermi frame to in astropy frame (J2000, etc.)
* plots the GBM NaI detectors at a given time for a given FOV
* determines if a astropy SkyCoord location is within an NaI's FOV
* creates interpolations over GBM quarternions and SC coordinates


In [1]:
%pylab inline
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord
import astropy.units as u
from gbmgeometry import *


Populating the interactive namespace from numpy and matplotlib


## Making an interpolation from TRIGDAT

### Getting the data

We can use GetGBMData to download the data

In [2]:
data = GetGBMData("080916009")


data.set_destination("/home/bbiltzing/test") # You can enter a folder here. If you want the CWD, you do not have to set


data.get_trigdat()



 [*********************100%***********************]  completed in 0.0 s

By default, GetGBMData will grad data from all detectors. However, one can set a subset to retrieve different data types

In [3]:
data.select_detectors()


data.get_rsp_cspec()

 [                       0%                       ]

### Interpolating
First let's create an interpolating object for a given TRIGDAT file (POSHIST files are also readable)


In [4]:
interp = PositionInterpolator(trigdat="glg_trigdat_all_bn080916009_v02.fit")

In [5]:
# In trigger times
print "Quaternions"
print interp.quaternion(0)
print interp.quaternion(10)
print
print "SC XYZ"
print interp.sc_pos(0)
print interp.sc_pos(10)





Quaternions
[0.09894184 0.81399423 0.56763536 0.07357984]
[0.09651158 0.81315938 0.56970097 0.06998621]

SC XYZ
[3184.75 5985.5  1456.75]
[3111.77432458 6015.91372132 1488.98009345]


## Single detector

One can look at a single detector which knows about it's orientation in the Fermi SC coordinates





In [6]:
na = NaIA(interp.quaternion(0))
print na.get_center()
print na.get_center().icrs #J2000
print na.get_center().galactic # Galactic
print 
print "Changing in time"
na.set_quaternion(interp.quaternion(100))
print na.get_center()
print na.get_center().icrs #J2000
print na.get_center().galactic # Galactic

TypeError: bad operand type for unary -: 'NoneType'

#### We can also go back into the GBMFrame 

In [7]:
center_j2000 = na.get_center().icrs
center_j2000

<SkyCoord (ICRS): (ra, dec) in deg
    (14.14827804, 51.13038764)>

In [9]:
center_j2000.transform_to(GBMFrame(quaternion=interp.quaternion(100.)))

NameError: name 'quaternion_1' is not defined

### Earth Centered Coordinates

The sc_pos are Earth centered coordinates (in km for trigdat and m for poshist) and can also be passed. It is a good idea to specify the units!




In [10]:
na = NaIA(interp.quaternion(0),interp.sc_pos(0)*u.km)
na.get_center()

<SkyCoord (GBMFrame: sc_pos_X=3184.75 km, sc_pos_Y=5985.5 km, sc_pos_Z=1456.75 km, quaternion_1=0.09894184023141861, quaternion_2=0.8139942288398743, quaternion_3=0.5676353573799133, quaternion_4=0.07357984036207199): (lon, lat) in deg
    (123.73, -0.42)>

## Working with the GBM class

Ideally, we want to know about many detectors. The GBM class performs operations on all detectors for ease of use. It also has plotting capabilities

In [13]:
myGBM = GBM(interp.quaternion(0),sc_pos=interp.sc_pos(0)*u.km,gbm_time=interp.utc(0))

myGBM.get_centers()



[<SkyCoord (GBMFrame: sc_pos_X=3184.75 km, sc_pos_Y=5985.5 km, sc_pos_Z=1456.75 km, quaternion_1=0.09894184023141861, quaternion_2=0.8139942288398743, quaternion_3=0.5676353573799133, quaternion_4=0.07357984036207199): (lon, lat) in deg
     (123.73, -0.42)>,
 <SkyCoord (GBMFrame: sc_pos_X=3184.75 km, sc_pos_Y=5985.5 km, sc_pos_Z=1456.75 km, quaternion_1=0.09894184023141861, quaternion_2=0.8139942288398743, quaternion_3=0.5676353573799133, quaternion_4=0.07357984036207199): (lon, lat) in deg
     (183.74, -0.32)>,
 <SkyCoord (GBMFrame: sc_pos_X=3184.75 km, sc_pos_Y=5985.5 km, sc_pos_Z=1456.75 km, quaternion_1=0.09894184023141861, quaternion_2=0.8139942288398743, quaternion_3=0.5676353573799133, quaternion_4=0.07357984036207199): (lon, lat) in deg
     (236.61, 0.03)>,
 <SkyCoord (GBMFrame: sc_pos_X=3184.75 km, sc_pos_Y=5985.5 km, sc_pos_Z=1456.75 km, quaternion_1=0.09894184023141861, quaternion_2=0.8139942288398743, quaternion_3=0.5676353573799133, quaternion_4=0.07357984036207199): (l

In [14]:
[x.icrs for x in myGBM.get_centers()]

[<SkyCoord (ICRS): (ra, dec) in deg
     (12.83264883, 51.9340734)>, <SkyCoord (ICRS): (ra, dec) in deg
     (344.24965886, -2.97248228)>, <SkyCoord (ICRS): (ra, dec) in deg
     (318.5160438, -51.24281778)>, <SkyCoord (ICRS): (ra, dec) in deg
     (44.56221395, 13.56776347)>, <SkyCoord (ICRS): (ra, dec) in deg
     (165.84085686, -0.42750824)>, <SkyCoord (ICRS): (ra, dec) in deg
     (345.84085686, 0.42750824)>, <SkyCoord (ICRS): (ra, dec) in deg
     (90.02053912, -5.02944102)>, <SkyCoord (ICRS): (ra, dec) in deg
     (106.9639318, 13.09532427)>, <SkyCoord (ICRS): (ra, dec) in deg
     (137.09793576, 52.86180786)>, <SkyCoord (ICRS): (ra, dec) in deg
     (121.3144115, -45.95986919)>, <SkyCoord (ICRS): (ra, dec) in deg
     (194.28783066, -52.03023808)>, <SkyCoord (ICRS): (ra, dec) in deg
     (164.65699738, 2.70662314)>, <SkyCoord (ICRS): (ra, dec) in deg
     (58.29477735, -33.54731071)>, <SkyCoord (ICRS): (ra, dec) in deg
     (28.32615836, -45.28243674)>]

### Plotting
We can look at the NaI view on the sky for a given FOV

In [7]:
myGBM.detector_plot(radius=60)

NameError: name 'myGBM' is not defined

In [ ]:
myGBM.detector_plot(radius=10,projection='ortho',lon_0=40)
myGBM.detector_plot(radius=10,projection='ortho',lon_0=0,lat_0=40,fignum=2)

### In Fermi GBM coodinates

We can also plot in Fermi GBM spacecraft coordinates

In [ ]:
myGBM.detector_plot(radius=60,fermi_frame=True)

In [ ]:
myGBM.detector_plot(radius=10,projection='ortho',lon_0=20,fermi_frame=True)
myGBM.detector_plot(radius=10,projection='ortho',lon_0=200,fermi_frame=True,fignum=3)
myGBM.detector_plot(radius=10,projection='ortho',lon_0=0,lat_0=40,fignum=2,fermi_frame=True)

### Capturing points on the sky

We can even see which detector's FOVs contain a point on the sky. We create a mock GRB SKycoord first.


In [16]:
grb = SkyCoord(ra=130.,dec=-45 ,frame='icrs', unit='deg')


myGBM.detector_plot(radius=60,
                    projection='moll',
                    good=True, # only plot NaIs that see the GRB
                    point=grb,
                    lon_0=110,lat_0=-0)


myGBM.detector_plot(radius=60,
                    projection='ortho',
                    good=True, # only plot NaIs that see the GRB
                    point=grb,
                    lon_0=180,lat_0=-40,fignum=2)

AttributeError: 'GBM' object has no attribute 'detector_plot'

### Looking at Earth occulted points on the sky

We can plot the points occulted by the Earth (assuming points 68.5 degrees form the Earth anti-zenith are hidden)


In [ ]:
myGBM.detector_plot(radius=10,show_earth=True,lon_0=90)

In [ ]:
myGBM.detector_plot(radius=10,lon_0=100,show_earth=True,projection='ortho')

In [ ]:
myGBM.detector_plot(radius=10,show_earth=True,lon_0=120,lat_0=-30,fermi_frame=True,projection='ortho')

## Source/Detector Separation
We can even look at the separation angles for the detectors and the source

In [ ]:
seps = myGBM.get_separation(grb)

seps.sort("Separation")

seps

## Examining Legal Detector Pairs

To see which detectors are valid, can look at the legal pairs map



In [ ]:
get_legal_pairs()